# Database Builder

This notebook builds the <b>SQLite database</b> that is used in this project in order to store all data obtained. <br>

Database and table are created only if they do not exist. If they exists, nothing will happen.
<hr>

#### Requirements

#### Database creation

In [ ]:
from lib.sqlite_utils import DBConnection 
from pathlib import Path
import os
 
Path("database").mkdir(parents=True, exist_ok=True)

conn=DBConnection(os.path.join('database','database.sqlite'))

## SQLITE DDL Table definitions

#### TABLE "organization"

In [5]:
conn.query("""CREATE TABLE IF NOT EXISTS organization ( -- ORGANIZATION: represents a GitHub organization
    url TEXT NOT NULL,  -- url of the GitHub organization
    section TEXT,       -- the section of the GitHub & Gov web page where the GitHub organization is located (if it is present)
    category TEXT,      -- the category of the GitHub & Gov web page where the GitHub organization is located (if it is present)
    name TEXT,          -- the name of the GitHub organization
    country TEXT,       -- if present, the country of the organization
    user_name TEXT,     -- the GitHub user_name of the GitHub organization
    location TEXT,      -- the location of the organization (obtained from GitHub metadata)
    repos INTEGER,      -- the number of public repositories of the organization
    followers INTEGER,  -- the number of followers of the organization
    created_at TEXT,    -- the creation date of the organization
    is_verified INTEGER,-- whether or not the organization is verified from GitHub (values in {0,1})
    members INTEGER,    -- the number of the members of the organization
    timestamp_created_at INTEGER, -- the timestamp of the creation date
    PRIMARY KEY (url) )
""")

[]

#### TABLE "repository"

In [ ]:
conn.query("""CREATE TABLE IF NOT EXISTS repository ( -- REPOSITORY: represents a GitHub repository
    name TEXT,  -- name of the GitHub repository
    organization TEXT,  -- the url of the GitHub organization that own the repository
    description TEXT,   -- description of the repository
    url TEXT NOT NULL,  -- the url of the repository
    is_fork INTEGER,    -- whether the repository is a fork or not (values in {0,1})
    size INTEGER,       -- the size of the repository in MB
    stars INTEGER,      -- the number of stars of the repository
    watcher INTEGER,     -- the number of watchers of the repository
    language TEXT,      -- the main language used within the repository
    forks INTEGER,      -- the number of times the repository has been forked
    open_issues INTEGER,-- the number of open issues of the repository
    created_at TEXT,    -- the creation date of the repository
    updated_at TEXT,    -- the date of the last update of the repository
    stored_at TEXT,     -- the date the repository has been stored in the database
    default_branch TEXT,-- the default branch of the repository
    timestamp_updated_at INTEGER, -- the timestamp of the last update date
    timestamp_created_at INTEGER, -- the timestamp of the creation date
    PRIMARY KEY (url),
    FOREIGN KEY (organization) REFERENCES organization(url) )
""")

#### TABLE "user"

In [ ]:
conn.query("""CREATE TABLE IF NOT EXISTS user ( -- USER: represents a GitHub user
    user_name TEXT,     -- the GitHub user_name of the user
    created_at TEXT,    -- the creation date of the GitHub user profile
    updated_at TEXT,    -- the date of the last update of the GitHub  user profile
    name TEXT,          -- the name of the GitHub user
    company TEXT,       -- the company of the user (obtained from GitHub metadata)
    location TEXT,      -- the location of the user (obtained from GitHub metadata)
    followers INTEGER,  -- the number of followers of the GitHub user
    following INTEGER,  -- the number of GitHub users the user follow
    email TEXT,         -- the email of the GitHub user
    bio TEXT,           -- The profile description of the user
    repos INTEGER,      -- The number of repositories of the user
    twitter_username TEXT,  -- If present, the twitter username of the user
    PRIMARY KEY (user_name) )
""")

####  TABLE "contributor"

In [ ]:
conn.query("""CREATE TABLE IF NOT EXISTS contributor ( -- CONTRIBUTOR: represents the relation between a REPOSITORY and a USER that contributes to it
    user_name TEXT,             -- the user_name of the user
    repository TEXT,            -- the repository url of the repository the user(user_name) contributes to
    contributions INTEGER,      -- the total number of contributions of the user to the repository
    rejected_pull_requests INTEGER,     -- the total number of rejected pull requests of the users during the repository lifetime (STORED BECAUSE: https://arxiv.org/abs/2103.03846)
    max_commit_in_a_day INTEGER,        --the maximum number of commit that the user have done in one day (STORED BECAUSE: https://arxiv.org/abs/2103.03846)
    first_commit_date TEXT,             -- the date of the first commit of the user (STORED BECAUSE: https://arxiv.org/abs/2103.03846)
    PRIMARY KEY (user_name, repository),
    FOREIGN KEY (user_name) REFERENCES user(user_name),
    FOREIGN KEY (repository) REFERENCES repository(url))
""")

#### TABLE "package"

In [ ]:
conn.query("""CREATE TABLE IF NOT EXISTS package ( -- PACKAGE: represents a package that come from a programming language ecosystem
    name TEXT,              -- the name of the package
    package_manager TEXT,   -- the package manager of the package (pypi,maven,npm,...)
    version TEXT,           -- the version of the package
    namespace TEXT,         -- the namespace of the package
    purl TEXT,              -- the purl (package url) of the package (SPEC: https://github.com/package-url/purl-spec)
    PRIMARY KEY (purl))
""")

#### TABLE "manifest_dependency"

In [ ]:
conn.query(""" CREATE TABLE IF NOT EXISTS manifest_dependency ( -- MANIFEST_DEPENDENCY: represents the dependency of a REPOSITORY on a PACKAGE (dependencies are collected from manifest files)
    repository TEXT,   -- the repository url 
    package TEXT,      -- the package purl
    PRIMARY KEY (repository,package),   
    FOREIGN KEY (package) REFERENCES package(purl),
    FOREIGN KEY (repository) REFERENCES repository(url))
""")

#### TABLE "parsed_dependency"

In [ ]:
conn.query(""" CREATE TABLE IF NOT EXISTS parsed_dependency (  -- PARSED_DEPENDENCY: represents the dependency of a REPOSITORY on a PACKAGE (dependencies are collected from import strings inside the code files)
    repository TEXT,    -- the repository url
    package TEXT,       -- the package purl
    PRIMARY KEY (repository,package),
    FOREIGN KEY (package) REFERENCES package(purl),
    FOREIGN KEY (repository) REFERENCES repository(url))
""")

#### TABLE "vulnerability"

In [ ]:
conn.query(""" CREATE TABLE IF NOT EXISTS vulnerability ( -- VULNERABILITY: represents a software vulnerability
    namespace TEXT,                 -- the namespace (type) of the id of the vulnerability (es. CVE, GHSA,...)
    id TEXT,                        -- the unique identifier of the vulnerability in its namespace
    base_score TEXT,                -- the base score of the vulnerability (https://nvd.nist.gov/vuln-metrics/cvss)
    severity TEXT,                  -- the base severity of the vulnerability (https://nvd.nist.gov/vuln-metrics/cvss)
    fixed_package_version TEXT,     -- the first package version where the vulnerability has been fixed (if it has been fixed)
    published_at TEXT,              -- the date of the publication of the vulnerability
    source TEXT,                    -- the collection procedure used (GRYPE, GRYPE_DB, OSV_API)
    other_sources TEXT,             -- another identifier of the vulnerability in another namespace
    url TEXT,                       -- the url to an explanation page of the vulnerability
    attack_vector TEXT,             -- the attack vector (https://nvd.nist.gov/vuln-metrics/cvss)
    attack_complexity TEXT,         -- the attack complexity (https://nvd.nist.gov/vuln-metrics/cvss)
    priviledge_required TEXT,       -- type of priviledge required for exploitation (https://nvd.nist.gov/vuln-metrics/cvss)
    user_interaction TEXT,          -- type of user interaction for exploitation (https://nvd.nist.gov/vuln-metrics/cvss)
    scope TEXT,                     -- scope (https://nvd.nist.gov/vuln-metrics/cvss)
    confidentiality_impact TEXT,    -- confidentiality impact (https://nvd.nist.gov/vuln-metrics/cvss)
    integrity_impact TEXT,          -- integrity impact (https://nvd.nist.gov/vuln-metrics/cvss)
    availability_impact TEXT,       -- availability impact (https://nvd.nist.gov/vuln-metrics/cvss)
    exploitability_score TEXT,      -- exploitability score (https://nvd.nist.gov/vuln-metrics/cvss)
    impact_score TEXT,              -- impact score (https://nvd.nist.gov/vuln-metrics/cvss)
    vector_string TEXT,             -- vector string (https://nvd.nist.gov/vuln-metrics/cvss)
    description TEXT,               -- description of the vulnerability
    PRIMARY KEY (id))
""")

#### TABLE "grype_potential_affection"

In [ ]:
conn.query(""" CREATE TABLE IF NOT EXISTS grype_potential_affection ( -- GRYPE_POTENTIAL_AFFECTION: the affection relation between a VULNERABILITY and a PACKAGE. Collected with Grype software
    vulnerability TEXT,     -- the vulnerability ID
    package TEXT,           -- the package purl
    PRIMARY KEY (vulnerability,package),
    FOREIGN KEY (package) REFERENCES package(purl),
    FOREIGN KEY (vulnerability) REFERENCES vulnerability(id))
""")

#### TABLE "grype_cpe_potential_affection"

In [ ]:
conn.query(""" CREATE TABLE IF NOT EXISTS grype_cpe_potential_affection ( -- GRYPE_CPE_POTENTIAL_AFFECTION: the affection relation between a VULNERABILITY and a PACKAGE. Collected with Grype software and --add-cpes-if-none parameter.
    vulnerability TEXT,     -- the vulnerability ID
    package TEXT,           -- the package purl
    PRIMARY KEY (vulnerability,package),
    FOREIGN KEY (package) REFERENCES package(purl),
    FOREIGN KEY (vulnerability) REFERENCES vulnerability(id))
""")

#### TABLE "osv_api_potential_affection"

In [ ]:
conn.query(""" CREATE TABLE IF NOT EXISTS osv_api_potential_affection ( -- OSV_API_POTENTIAL_AFFECTION: the affection relation between a VULNERABILITY and a PACKAGE. Collected with OSV API.
    vulnerability TEXT,     -- the vulnerability ID
    package TEXT,           -- the package purl
    PRIMARY KEY (vulnerability,package),
    FOREIGN KEY (package) REFERENCES package(purl),
    FOREIGN KEY (vulnerability) REFERENCES vulnerability(id))
""")

In [ ]:
conn.close()